<a href="https://colab.research.google.com/github/suhasms369/AI_Lab_1BM23CS346/blob/main/Firstorderlogic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
facts = {
    "American(Robert)",
    "Enemy(CountryA, America)",
    "Missile(M1)",
    "Has(CountryA, M1)",
    "Sold(Robert, CountryA, M1)"
}

rules = [
    ("Enemy(X, America)", "Hostile(X)"),
    ("American(Robert) & Sold(Robert, Y, Z) & Hostile(Y) & Weapon(Z)", "Criminal(Robert)"),
    ("Missile(Z)", "Weapon(Z)")
]

import re

def forward_chaining(facts, rules, goal):
    new_facts = set(facts)
    while True:
        added = False
        for premise, conclusion in rules:
            premises = [p.strip() for p in premise.split('&')]
            substitutions = [{}]
            for p in premises:
                new_substitutions = []
                for sub in substitutions:
                    p_inst = substitute(p, sub)
                    matches = match_fact(p_inst, new_facts)
                    for m in matches:
                        merged = merge_substitutions(sub, m)
                        if merged is not None:
                            new_substitutions.append(merged)
                substitutions = new_substitutions
            for sub in substitutions:
                conclusion_inst = substitute(conclusion, sub)
                if conclusion_inst not in new_facts:
                    new_facts.add(conclusion_inst)
                    print(f"Inferred: {conclusion_inst}")
                    added = True
                    if conclusion_inst == goal:
                        print("Goal reached!")
                        return True
        if not added:
            break
    return goal in new_facts

def substitute(expr, sub):
    for var, val in sub.items():
        expr = re.sub(r'\b' + re.escape(var) + r'\b', val, expr)
    return expr

def match_fact(premise, facts):
    pattern = re.compile(r"(\w+)\(([^()]+)\)")
    m = pattern.match(premise)
    if not m:
        return []
    pred = m.group(1)
    args = [a.strip() for a in m.group(2).split(",")]
    matches = []
    for f in facts:
        fm = pattern.match(f)
        if not fm:
            continue
        fpred = fm.group(1)
        fargs = [a.strip() for a in fm.group(2).split(",")]
        if fpred != pred or len(fargs) != len(args):
            continue
        sub = {}
        failed = False
        for a, fa in zip(args, fargs):
            if is_variable(a):
                if a in sub and sub[a] != fa:
                    failed = True
                    break
                sub[a] = fa
            else:
                if a != fa:
                    failed = True
                    break
        if not failed:
            matches.append(sub)
    return matches

def merge_substitutions(s1, s2):
    s = s1.copy()
    for k, v in s2.items():
        if k in s and s[k] != v:
            return None
        s[k] = v
    return s

def is_variable(x):
    return bool(re.match(r'^[A-Z][A-Za-z0-9_]*$', x))

goal = "Criminal(Robert)"

if forward_chaining(facts, rules, goal):
    print("Robert is criminal: Proven")
else:
    print("Could not prove Robert is criminal")

print("By:\n   Suhas MS\n   1BM23CS346")


Inferred: Hostile(CountryA)
Inferred: Weapon(M1)
Inferred: Criminal(Robert)
Goal reached!
Robert is criminal: Proven
By:
   Suhas MS
   1BM23CS346
